<a href="https://colab.research.google.com/github/ssanchezgoe/curso_deep_learning_economia/blob/main/NBs_Google_Colab/DL_S14_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Curso Deep Learning: Economía</h1>

## S14: Redes neuronales convolucionales

# Redes neuronales convolucionales (CNN)

Las redes neuronales convolucionales (CNN) surgieron del estudio de la corteza visual del cerebro, y se han utilizado en el reconocimiento de imágenes desde la década de 1980. En los últimos años, gracias al aumento en el poder computacional, la cantidad de datos de entrenamiento disponibles y los trucos presentados en la clase anteriro para entrenar redes profundas, las CNN han logrado alcanzar un rendimiento sobrehumano en algunas tareas visuales complejas. Impulsan servicios de búsqueda de imágenes, autos que se manejan solos, sistemas automáticos de clasificación de video y más. Además, las CNN no se limitan a la percepción visual: también tienen éxito en muchas otras tareas, como el reconocimiento de voz o el procesamiento del lenguaje natural (PNL).

La diferencia fundamental entre una capa densamente conectada y una capa de convolución (convolutional layer ) es que las capas densas aprenden patrones globales en su espacio de características de entrada (por ejemplo, para un dígito MNIST, patrones que involucran todos los píxeles), mientras que las capas de convolución aprenden patrones locales, enn el caso de imágenes, patrones encontrados en pequeñas ventanas de las caracteristicas de entrada. 

<p><img alt="Colaboratory logo" height="300px" src="https://i.imgur.com/qdvojdR.png" align="center" hspace="10px" vspace="0px"></p> 

Esta característica clave otorga a las capas convolucionales dos propiedades interesantes:
* Los patrones que aprenden son invariantes de translacion. Después de aprender un cierto patrón en la esquina inferior derecha de una imagen, una CNN puede reconocerlo en cualquier lugar, por ejemplo, en la esquina superior izquierda. Una red densamente conectada tendría que aprender el patrón nuevamente si apareciera en una nueva ubicación. Esto hace que los datos de las CNN sean eficientes cuando se procesan imágenes (porque el mundo visual es fundamentalmente invariante ante translacion).

* Pueden aprender jerarquías espaciales de patrones: una primera capa de convolución aprenderá pequeños patrones locales como los bordes, una segunda capa de convolución aprenderá patrones más grandes hechos de las características de las primeras capas, y así sucesivamente. Esto permite que las CNN aprendan eficientemente conceptos visuales cada vez más complejos y abstractos (porque el mundo visual es fundamentalmente jerárquico espacialmente).

<p><img alt="Colaboratory logo" height="300px" src="https://i.imgur.com/FaeuJas.png" align="center" hspace="10px" vspace="0px"></p> 

Ahora que tenemos una idea de que son las CNN pasemos a ver el elemento que hace tan especiales a las CNN, esto es , las capas convolucionales. 

## Convolutional layer
Una capa convolucional tiene por lo general tres etapas como se muestra en la figura 

<p><img alt="Colaboratory logo" height="450px" src="https://i.imgur.com/PyoAVmM.png" align="center" hspace="10px" vspace="0px"></p> 

Vemos cada una de estas etapas con un poco mas de detalle 

## Stage 1:  Convolución

La convolución (en el contexto las redes neuronales convolucionales) es básicamente una correlación cruzada (sin embargo la seguiremos llamando convolución), la cual, en procesamiento de señales, es una medida de la similitud entre dos señales, frecuentemente usada para encontrar características relevantes en una señal desconocida por medio de la comparación con otra que sí se conoce.
Matematicamente la correlacion cruzada (en el caso discreto) se defien como:

\begin{equation}
S[t] = (X * W)(t) = \sum_a X[a]W[t+a]
\end{equation}

para entender mejor el concepto de convolución veamos un ejemplo simple usando dos señales. 

### Ejemplo : dos señales 1-dimensional

Para ver de forma mas detalla como se hacen animacion con matplotlib en google colab puede visitar el link: https://colab.research.google.com/drive/1lnl5UPFWVPrryaZZgEzd0theI6S94c3X#scrollTo=OEwd0xc5eGz9

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
#import seaborn as sns; sns.set()

In [ ]:
# Primero configure la figura, el eje y el elemento de la trama que queremos animar
fig, ax = plt.subplots(figsize=(13,8) )
plt.close()

y1 = np.array([0.1,0.2,-0.1,4.1,-2,1.5,-0.1])
x1=np.arange(1,len(y1)+1)
ax.plot(x1,y1+7,'o-')

ax.set_xlim(( -7, 15))
ax.set_ylim((-3, 12))
ax.set_yticks([])
ax.set_xticks([])
line, = ax.plot([], [], 'o-r')

In [ ]:
# función de inicialización: traza el fondo de cada cuadro
def init():
    line.set_data([], [])
    return (line,)

# función de animación
def animate(i):
  i=i-6
  y2 = np.array([0.1,4,-2.2,1.6,0.1,0.1,0.2])
  x=np.arange(1,len(y2)+1)+i
  line.set_data(x, y2)
  ax.set_title('cross correlation=%.3f' %(np.correlate(y1,y2,mode='full'))[6+i], fontsize=20)
  for t in ax.texts:
    t.set_visible(False)

  for i in range(len(x)):
    ax.text(x1[i], y1[i]+7, str(y1[i]))
    ax.text(x[i], y2[i], str(y2[i]))

    
  return (line,)

In [ ]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                             frames=7+6, interval=2000, blit=True)

rc('animation', html='jshtml')
anim

como se puede observar , el valor más grande de correlación cruzada es $23.18$. Si comparamos la forma de las señales justo para este valor de correlación vemos que la forma de las señales es bastante parecida. Vemos entonces que la correlación cruzada es un indicador de la similitud de la forma ( de las características ) de estas dos señales.  

### Caso extendido 

En el contexto de las CNNs, estas señales son multidimensionales y no solo 1-dimensional como en el ejemplo que acabamos de ver. Consideremos por ejemplo el caso de una imagen 2-dimensional (supongamos una imagen a blanco y negro), para este caso nuestra ecuación para la convolución(correlación cruzada) la siguiente forma:


\begin{equation}
S[i,j] = (I * K)[i,j] = \sum_m \sum_n I[m,n]W[m+i,n+j]
\end{equation}

Donde en el contexto de las CNNs, la señal $K$ es conocida como Kernel. En la ecuación anterior estamos haciendo la convolución de la imagen $I$ con el Kernel $K$.

veamos un ejemplo práctico de esto para entender un poco mejor qué es lo que sucede

#### Ejemplo: imagen 2-dimensional 

Consideremos la siguiente imagen y el sigueinte kernel

In [ ]:
I = np.array([[1,1,1,0,0], [0,1,1,1,0], [0,0,1,1,1], [0,0,1,1,0], [0,1,1,0,0]])
K = np.array([[1,0,1],[0,1,0],[1,0,1]])

In [ ]:
print(I)

In [ ]:
print(K)

Ahora usemos la libreria de Scipy para realizar la convolucion 

In [ ]:
from scipy.signal import convolve2d

In [ ]:
convolve2d(I,K, mode='valid')




![](http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif)

Este tipo de convolución es es conocida como convolucion valid.

Realizando esta operación de convolución entre $I$ y $K$, podemos capturar algunas de las características de la imagen que son similares a las de nuestro kernel. Para apreciar mejor esta idea , veamos un ejemplo con una imagen más grande.



#### Convolucion en imagen mas grande

Por ahora consideremos una imagen a blanco y negro , sin embargo más adelante veremos que las imágenes pueden venir con más canales (pro ejemplo los ampliamente conocidos RGB)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy.signal import convolve2d

In [ ]:
image= mpimg.imread('https://i.imgur.com/R2mS8Oh.png')

In [ ]:
image.shape

In [ ]:
K=np.array([[1,1,1],
           [0,0,0],
           [-1,-1,-1]])

In [ ]:
fig , ax = plt.subplots(2,3, figsize=(18,15), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.05, wspace=0.1))
ax[0,0].imshow(image, cmap='Greys' )
ax[0,1].imshow(K,cmap='Greys')
ax[0,2].imshow(K.T,cmap='Greys')
ax[1,1].imshow(convolve2d(image,K, mode='valid'), cmap='Greys')
ax[1,2].imshow(convolve2d(image,K.T, mode='valid'), cmap='Greys')
ax[1,0].imshow(convolve2d(image,K.T, mode='valid')+convolve2d(image,K, mode='valid'), cmap='Greys')

En las CNNs los kernels son aprendidos por nuestro algoritmo, es decir,  basados en las predicciones que queremos realizar , le decimos a nuestro modelo que encuentre cuál es el mejor kernel para dicha tare.  

### Efectos de borde, y strides
como se vio en el ejemplo anterior, después de realizar la operación de convolución la imagen redujo su tamaño. En general, el ancho y alto de la imagen de salida pueden diferir del ancho y alto de la imagen de entrada debido a dos razones: 

*  Efectos de borde, que se pueden contrarrestar usando el metodo llamado padding.

*  El uso de strides


#### Efectos de borde y padding 

Consideremos el ejemplo de nuestra imagen de 5x5 píxeles, y un kernel de 3x3.
<p><img alt="Colaboratory logo" height="300px" src="https://i.imgur.com/uyepS9e.png" align="center" hspace="10px" vspace="0px"></p> 


si queremos que el tamaño de nuestra imagen de salida (esto es, después de aplicar la operación de convolución con el kernel) tenga el mismo tamaño de nuestra imagen de entrada podemos usar la técnica llamada padding. Padding consiste en agregar un número apropiado de filas y columnas a cada lado de la imagen para que así la imagen de salida tenga las mismas dimensiones de la imagen de entrada. Para un kernel 3×3, agrega una columna a la derecha, una columna a la izquierda, una fila en la parte superior y una fila en la parte inferior.

<p><img alt="Colaboratory logo" height="250px" src="https://i.imgur.com/p1MdvHF.png" align="center" hspace="10px" vspace="0px"></p> 

La convolución que consta de padding y luego convolución , es conocida como convolución “Same”. Veamos un ejemplo de esto usando scipy 


In [ ]:
convolve2d(I,K, mode='same')

Usualmente el padding se realiza agregando columnas y filas de ceros

<p><img alt="Colaboratory logo" height="350px" src="https://i.imgur.com/5BoEHiY.png" align="center" hspace="10px" vspace="0px"></p> 



#### Strides 

Como vimos en el ejemplo de la imagen 5x5 , cuando desplazamos nuestro kernel en la imagen para realizar la operación de convolución , nos desplazamos solo una columna o solo una fila, sin embargo estos desplazamientos son un parámetro de la convolución llamado stride y en general puede ser diferente de uno. 
<p><img alt="Colaboratory logo" height="350px" src="
https://i.imgur.com/vMGjDk9.png" align="center" hspace="10px" vspace="0px"></p>

Debido a fijar los stides diferente de uno , la imagen puede verse reducida después de la operación de convolución.


## Stage 2: Detector Stage (activation stage)

Esta etapa es similar a la que ya conocemos de las DNN, se trata de aplicar una transformación no lineal (funciones de activación) tales como Relu, Tanh, etc.



## Stage 3: Pooling

En la etapa de Polling vamos a calcular un resumen estadístico de nuestra imagen una vez a pasado por las dos etapas anteriores ( esto es , convolution  y detector stage). Hay varias razones para realizar esto: 

* Reducir la imagen de entrada para reducir la carga computacional, el uso de memoria y el número de parámetros (lo que limita el riesgo de overfitting).

* Introducir cierto nivel de invariancia a pequeñas traslaciones.

Hay diferentes formas de hacer pooling , entre las más conocidas están Max Polling ( The maximum of a rectangular neighborhood) 

<p><img alt="Colaboratory logo" height="300px" src="https://i.imgur.com/BW48gCv.png" align="center" hspace="10px" vspace="0px"></p>

Ahora tenemos todas las herramientas para construir nuestras CNN


## Forma mas gerenal 
Hasta ahora, por simplicidad, hemos representado la salida de cada capa convolucional como una delgada capa 2D, pero en realidad una capa convolucional tiene múltiples Kernels (filtros), y genera un mapa de características por filtro, por lo que es representado con mayor precisión en 3D.

Además, las imágenes de entrada también se componen de múltiples subcapas: una por canal de color. Normalmente hay tres: rojo, verde y azul (RGB). Las imágenes en escala de grises tienen solo un canal, pero algunas imágenes pueden tener mucho más, por ejemplo, imágenes satelitales que capturan frecuencias de luz adicionales (como infrarrojo).
<p><img alt="Colaboratory logo" height="400px" src="https://i.imgur.com/OuDTED7.png" align="center" hspace="10px" vspace="0px"></p>


## La arquitectura tipica de las CNN es como se muestra en la figura de abajo 

<p><img alt="Colaboratory logo" height="300px" src="https://i.imgur.com/BqlLRkJ.png" align="center" hspace="10px" vspace="0px"></p>






In [ ]:
from tensorflow import keras

In [ ]:
keras.backend.clear_session()

### **Implementación de una red convolusional en Keras**
La construcción de nuestra red se da de manera similar a como se ha venido realizando, solo que para este caso debemos añadir capas convolucionales, es decir que realicen el proceso de convolución. Keras cuenta con diferentes tipos de capas convolucionales, esta será definida de acuerdo a nuestras necesidades, estás se pueden ver en el siguiente [link](https://keras.io/layers/convolutional/). 

Veamos como sería la implementación para el caso de una **convolución2D**. Al crear nuestra red se añade la capa de la siguiente manera:



In [ ]:
mod=keras.models.Sequential([
      keras.layers.Conv2D(filters=64,kernel_size=3,strides=(1,1),padding='valid',activation='relu',use_bias=True,kernel_initializer='he_uniform', bias_initializer='zeros')
])

En la línea de código anterior de forma didáctica se construyó una capa convolucional en la forma que se hace en keras, allí solo tuvimos en cuenta algunos argumentos. Esta capa crea un kernel de convolución que se convoluciona  (valga la redundancia) con capa de entrada para producir un tensor como salida. Dado el caso de que esta sea nuestra capa de entrada, recordemos debemos entrar la forma de neustros datos, donde este será la forma de neustra matriz más la componente que nos dará el número de canales que se tenga. 

Los argumentos en la anterior capa hacen referencia a :
* **Filters:** Entrada para el número de filtro que usará nuestra capa, valor entero.
* **kernel_size:** Recibe una tupla como entrada en la que le daremos el valor de la altura y el ancho de nuestro filtro (kernel), si le damos un entero como en este caso, keras interpretará como que tiene ese mismo valor en la altura y el ancho.
* **Strides:** Recibe una tupla como entrada en la que específicamos los strides, en dirección horizontal y vertical.

* **Padding:** recibe como argumento 'valid' o 'same', donde en la primera no tendremos padding y con la segunda nuestra salida tendrá las mismas dimensiones que nuestra entrada.
* **Activation:** Función de activación, juega el papel que ya hemos visto en las anteriores clase.
* **use_bias:** Me permite definir si mi capa tendrá un vector de preferencia o sesgo (bias vector).

* **initializers:** Forma de inicializar mis parámetros, ya sea del kernel o del vector de preferencias.

Para este tipo de capa tenemos unos cuantos argumentos más que se pueden consultar en el siguiente [link](https://keras.io/layers/convolutional/)

Veamos como se da la implementación de una capa convolucional en una red, para esto haremos un ejemplo en el cual usando el **fashion mnist** dataset compararemos las diferencias entre la implementación con una red densa multicapa y una red convoluvional. Para eso lo primero que haremos es impotar las librerías necesarias y cargar el dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
%tensorflow_version 2.x

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full.shape

Como vimos en clases pasadas, un punto importante es la normalización de los datos, en este caso nuestros pixeles, esto es lo que realizaremos en el siguiente paso

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

En este dataset, recordemos que las etiquetas estan dadas por enteros del 0-9, así que para darle un nombre como tal a cada una debemos tener el siguiente arreglo con las clases.

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
np.unique(y_train)

In [ ]:
class_names[y_train[0]]

In [ ]:
print('X_valid:',X_valid.shape,'\t','X_test:',X_test.shape,'\t','X_train:',X_train.shape)

Veamos algunos de los elementos que podemos encontrar en nuestro dataset

In [ ]:
fig , ax =  plt.subplots(3,10, figsize=(15,5))
for i , ax in enumerate(ax.flat):
  ax.imshow(X_train[i], cmap='binary')
  ax.set_axis_off()
  ax.set_title(class_names[y_train[i]])

Veamos como sería la implementación de una red densa profunda, la cual ya hemos visto en clases pasadas.

In [ ]:
keras.backend.clear_session()

In [ ]:
model = keras.models.Sequential();
model.add(keras.layers.Flatten(input_shape=[28, 28]));
model.add(keras.layers.Dense(300, activation="relu"));
model.add(keras.layers.Dense(100, activation="relu"));
model.add(keras.layers.Dense(10, activation="softmax"));

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=["accuracy"])

In [ ]:
history =  model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Veamos el comportamiento en la curva de validación.

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,10))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
model.evaluate(X_test,y_test)

Ahora veamos una implementación de una red neuronal en la cual tenemos dos capas convolucionales en las cuales la Detector stage usaremos como activación la **relu**.

In [ ]:
keras.backend.clear_session()

In [ ]:
model2= keras.models.Sequential();
model2.add(keras.layers.Conv2D(64, kernel_size=3, activation="relu", input_shape=(28,28,1)));
model2.add(keras.layers.Conv2D(32, kernel_size=3, activation="relu"));
model2.add(keras.layers.Flatten());
model2.add(keras.layers.Dense(100, activation="relu"));
model2.add(keras.layers.Dense(10, activation="softmax"));

**Opcional:** Podemos usar el Early stopping, para que nuestro modelo se detenga cuando empiece a aumentar el valor de la función de perdida en el conjunto de validación.

In [ ]:
from keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='val_loss',patience=1)

Usamos el mismo compilador anterior.

In [ ]:
model2.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=["accuracy"])

En este caso como usaremos imagenes en escala de grises, debemos hacer un reshape a nuestros datos, de forma tal que los entreguemos en la forma que nuestra red los espera, donde la primer componente hace referencia al número de imagenes, las dos siguientes a las formas de estas (28$\times$28), y finalamente la última hace referencia a los canales.

In [ ]:
X_train1=X_train.reshape(55000,28,28,1)
X_valid1=X_valid.reshape(5000,28,28,1)

In [ ]:
history2=  model2.fit(X_train1, y_train, epochs=20, validation_data=(X_valid1, y_valid),callbacks=[es])

In [ ]:
pd.DataFrame(history2.history).plot(figsize=(10,10))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

Podemos ver como una red con una capa convolucional mejora bastante mi modelo, en un número de épocas menor.

### **Pooling en keras**
Keras además nos permite tener capas de pooling en nuestras redes convolucionales, las cuales se pueden definir de la siguiente manera:

In [ ]:
mod=keras.models.Sequential([
      keras.layers.Conv2D(filters=64,kernel_size=3,strides=(1,1),padding='valid',activation='relu',use_bias=True,kernel_initializer='he_uniform', bias_initializer='zeros'),
      keras.layers.MaxPooling2D(pool_size=(2,2),strides=None,padding='valid')
])

Donde vemos tenemos la capa que explicamos al inicio de esta sección (convolucional) y le añadimos una capa de pooling, donde los argumentos son el tamaño del pool, el stride y el padding. Para más información sobre capas Pooling ir al siguiente [link](https://keras.io/layers/pooling/). Veamos que pasará en el mnist dataset en una red en la cual añadimos una capa de pooling

In [ ]:
keras.backend.clear_session()
model3= keras.models.Sequential();
model3.add(keras.layers.Conv2D(64, kernel_size=3, activation="relu", input_shape=(28,28,1)));
model3.add(keras.layers.Conv2D(32, kernel_size=3, activation="relu"));
model3.add(keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)));
model3.add(keras.layers.Flatten());
model3.add(keras.layers.Dense(100, activation="relu"));
model3.add(keras.layers.Dense(10, activation="softmax"));

In [ ]:
model3.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=["accuracy"])

In [ ]:
history3=  model3.fit(X_train1, y_train, epochs=20, validation_data=(X_valid1, y_valid),callbacks=[es])

In [ ]:
pd.DataFrame(history3.history).plot(figsize=(10,10))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

Donde podemos ver que la diferencia al añadirle esta capa no es mucha para este caso. Sin embargo estas capas son de gran ayuda en ciertos casos.